In [259]:
from pandas.io.json import json_normalize
import json
import urllib.request
import pandas as pd


# API Key
# replace YOUR_API_KEY with the API key you got from sec-api.io after sign up
TOKEN = "085838d0e1b52d643b85150a1055b00cbd181982756cef3b17207071d2952ce0"
# API endpoint
API = "https://api.sec-api.io?token=" + TOKEN


def get_query(ticker):
    # Define the filter parameters
    filter = "ticker:" + ticker + " AND formType:\"S-1\""

    # Start with the first filing. Increase it when paginating.
    # Set to 10000 if you want to fetch the next batch of filings. Set to 20000 for the next and so on.
    start = 0
    # Return 10,000 filings per API call
    size = 100
    # Sort in descending order by filedAt
    sort = [{"filedAt": {"order": "desc"}}]

    payload = {
        "query": {"query_string": {"query": filter}},
        "from": start,
        "size": size,
        "sort": [
            {
                "filedAt": {
                    "order": "desc"
                }
            }
        ]
    }

    # Format your payload to JSON bytes
    jsondata = json.dumps(payload)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes

    # Instantiate the request
    req = urllib.request.Request(API)

    # Set the correct HTTP header: Content-Type = application/json
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    # Set the correct length of your request
    req.add_header('Content-Length', len(jsondataasbytes))

    # Send the request to the API
    response = urllib.request.urlopen(req, jsondataasbytes)

    # Read the response
    res_body = response.read()
    # Transform the response into JSON
    filingsJson = json.loads(res_body.decode("utf-8"))

    return filingsJson['filings']



In [232]:
def get_data(tickers_df):
    for i in tickers_df.iterrows():
        print(i[1]['IPO name'])
        print(i[1]['Ticker'])
        ticker = i[1]['Ticker']
        result = (get_query(str(ticker)))
        df = json_normalize(result)
        if len(df) != 0:
            df = df[df['formType'] == "S-1"]
            if len(df) != 0:
                df = df.sort_values("filedAt", ascending=False).reset_index(drop=True)
                secLink = df['linkToHtml'][0]
                index = i[0]
                print(index)
                tickers_df.iloc[index, 6] = secLink
                print(secLink)
    return tickers_df

In [218]:
pd.set_option('display.max_colwidth', -1)

In [219]:
tickers= pd.read_excel('internet-IPOs.xlsx')


In [220]:
tickers = tickers.sort_values("Offer date", ascending=False).reset_index(drop=True)

In [221]:
tickers['S1Link'] = ''

In [222]:
tickers.head()

,IPO name,Offer date,SIC,Ticker,Unit,Offer price,S1Link
0,Stoneco Ltd,20181025,7372,STNE,1,24.0,
1,Anaplan Inc,20181012,7376,PLAN,1,17.0,
2,Elastic NV,20181005,7379,ESTC,1,36.0,
3,Upwork Inc,20181003,7374,UPWK,1,15.0,
4,SVMK Inc,20180926,7374,SVMK,1,12.0,


In [223]:
def splitDataFrameIntoSmaller(df, chunkSize = 80): 
    listOfDf = list()
    numberChunks = len(df) // chunkSize + 1
    for i in range(numberChunks):
        listOfDf.append(df[i*chunkSize:(i+1)*chunkSize])
    return listOfDf


In [230]:
list_of_df = splitDataFrameIntoSmaller(tickers)

In [226]:
df_0 = get_data(list_of_df[0])

Stoneco Ltd
STNE
Anaplan Inc
PLAN


/Users/sharan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


https://www.sec.gov/Archives/edgar/data/1540755/0001193125-18-274158-index.htm
Elastic NV
ESTC


/Users/sharan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


https://www.sec.gov/Archives/edgar/data/1707753/0001193125-18-266861-index.htm
Upwork Inc
UPWK


...

https://www.sec.gov/Archives/edgar/data/1477200/0001047469-14-000286-index.htm
FireEye Inc
FEYE
RetailMeNot Inc
SALE
CDW Corp
CDW


/Users/sharan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


https://www.sec.gov/Archives/edgar/data/1402057/0001193125-13-122411-index.htm
Gogo Inc
GOGO


/Users/sharan/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


https://www.sec.gov/Archives/edgar/data/1537054/0001193125-11-351260-index.htm
LightInTheBox Hldg Co Ltd
LITB
QIWI PLC
QIWI
Professional Diversity Network
IPDN
YY Inc
YY
Shutterstock Inc
SSTK
LifeLock Inc
LOCK
Qualys Inc
QLYS


In [227]:
list_of_df[0].head()

,IPO name,Offer date,SIC,Ticker,Unit,Offer price,S1Link
0,Stoneco Ltd,20181025,7372,STNE,1,24.0,
1,Anaplan Inc,20181012,7376,PLAN,1,17.0,https://www.sec.gov/Archives/edgar/data/1540755/0001193125-18-274158-index.htm
2,Elastic NV,20181005,7379,ESTC,1,36.0,https://www.sec.gov/Archives/edgar/data/1707753/0001193125-18-266861-index.htm
3,Upwork Inc,20181003,7374,UPWK,1,15.0,https://www.sec.gov/Archives/edgar/data/1627475/0001193125-18-267594-index.htm
4,SVMK Inc,20180926,7374,SVMK,1,12.0,https://www.sec.gov/Archives/edgar/data/1739936/0001193125-18-261892-index.htm


In [236]:
df_1 = get_data(list_of_df[1].reset_index(drop=True))

Trulia Inc
TRLA
KAYAK Software Corp
KYAK
1
https://www.sec.gov/Archives/edgar/data/1312928/0001193125-10-262521-index.htm
ServiceNow Inc
NOW
2
https://www.sec.gov/Archives/edgar/data/1373715/0001193125-12-143517-index.htm
Facebook Inc
FB
3
https://www.sec.gov/Archives/edgar/data/1326801/0001193125-12-034517-index.htm
Infoblox Inc
BLOX
4
https://www.sec.gov/Archives/edgar/data/1223862/0001193125-12-395987-index.htm
Proofpoint Inc
PFPT
5
https://www.sec.gov/Archives/edgar/data/1212458/0001047469-11-010082-index.htm
CafePress Inc
PRSS
6
https://www.sec.gov/Archives/edgar/data/1117733/0001193125-11-162782-index.htm
Millennial Media Inc
MM
7
https://www.sec.gov/Archives/edgar/data/1372375/0001047469-12-009489-index.htm
Vantiv Inc
VNTV
Demandware Inc
DWRE
9
https://www.sec.gov/Archives/edgar/data/1301031/0001193125-11-189260-index.htm
Yelp Inc
YELP
10
https://www.sec.gov/Archives/edgar/data/1345016/0001193125-11-315562-index.htm
Bazaarvoice Inc
BV
11
https://www.sec.gov/Archives/edgar/data/1

In [242]:
df_2 = get_data(list_of_df[2].reset_index(drop=True))

WNS(Holdings)Ltd
WNS
GMarket Inc
GMKT
Omniture Inc
OMTR
2
https://www.sec.gov/Archives/edgar/data/1357525/0000891020-07-000160-index.htm
LoopNet Inc
LOOP
3
https://www.sec.gov/Archives/edgar/data/1504678/0001165527-10-000980-index.htm
Vonage Hldgs Corp
VG
4
https://www.sec.gov/Archives/edgar/data/1272830/0001047469-06-001567-index.htm
Liquidity Services Inc
LQDT
5
https://www.sec.gov/Archives/edgar/data/1235468/0001104659-07-009834-index.htm
NightHawk Radiology Holdings
NHWK
6
https://www.sec.gov/Archives/edgar/data/1292470/0001193125-06-203965-index.htm
Digital Music Group Inc
DMGI
IncrediMail Ltd
MAIL
Traffic.com Inc
TRFC
9
https://www.sec.gov/Archives/edgar/data/1097503/0001047469-05-022277-index.htm
DealerTrack Holdings
TRAK
10
https://www.sec.gov/Archives/edgar/data/1333513/0000950123-05-008991-index.htm
Vocus
VOCS
11
https://www.sec.gov/Archives/edgar/data/1329919/0000950133-05-002680-index.htm
Website Pros Inc
WSPI
VistaPrint Limited
VPRT
WebMD Health Corp
WBMD
14
https://www.se

In [243]:
df_2

,IPO name,Offer date,SIC,Ticker,Unit,Offer price,S1Link
0,WNS(Holdings)Ltd,20060725,7389,WNS,1,20.000,
1,GMarket Inc,20060628,5961,GMKT,1,15.250,
2,Omniture Inc,20060627,7372,OMTR,1,6.500,https://www.sec.gov/Archives/edgar/data/1357525/0000891020-07-000160-index.htm
3,LoopNet Inc,20060606,6531,LOOP,1,12.000,https://www.sec.gov/Archives/edgar/data/1504678/0001165527-10-000980-index.htm
4,Vonage Hldgs Corp,20060523,4813,VG,1,17.000,https://www.sec.gov/Archives/edgar/data/1272830/0001047469-06-001567-index.htm
5,Liquidity Services Inc,20060222,7375,LQDT,1,10.000,https://www.sec.gov/Archives/edgar/data/1235468/0001104659-07-009834-index.htm
6,NightHawk Radiology Holdings,20060208,8071,NHWK,1,16.000,https://www.sec.gov/Archives/edgar/data/1292470/0001193125-06-203965-index.htm
7,Digital Music Group Inc,20060202,6794,DMGI,1,9.750,
8,IncrediMail Ltd,20060130,7371,MAIL,1,7.500,
9,Traffic.com Inc,20060125,7375,TRFC,1,12.000,https://www.sec.gov/Archives/edgar/data/1097503/0001047469-05-022277-index.htm


In [246]:
df_3 = get_data(list_of_df[3].reset_index(drop=True))

SynQuest Inc
SYNQ
Quintalinux Ltd
QLNX
Equinix Inc
EQIX
2
https://www.sec.gov/Archives/edgar/data/1101239/0001012870-00-003459-index.htm
RadView Software Ltd
RADV
Evolve Software Inc
EVLV
Mind C.T.I. Ltd
MNDO
America Online Latin America
AOLA
6
https://www.sec.gov/Archives/edgar/data/1100395/0000950130-00-000161-index.htm
ScreamingMedia Inc
SCRM
Resonate Inc
RSNT
iAsia Works Inc
IAWK
Floware Wireless Systems Ltd
FLRE
Convergent Group Corp
CVGP
Webex Communications Inc
WEBX
12
https://www.sec.gov/Archives/edgar/data/1109935/0000891618-00-001916-index.htm
ValiCert Inc
VLCT
Mainspring Inc
MSPR
Lexent Inc
LXNT
Corvis Corp
CORV
Interland Inc
ILND
Blue Martini Software Inc
BLUE
Evoke Communications Inc
EVOK
19
https://www.sec.gov/Archives/edgar/data/1403708/0001193125-13-235439-index.htm
Talarian Corp
TALR
Support.com Inc
SPRT
21
https://www.sec.gov/Archives/edgar/data/1104855/0001012870-00-000859-index.htm
Network Engines Inc
NENG
22
https://www.sec.gov/Archives/edgar/data/1110903/000092701

In [249]:
df_4 = get_data(list_of_df[4].reset_index(drop=True))

Riverdeep Interactive Learning
RVDP
HomeGrocer.com
HOMG
iprint.com Inc
IPRT
Versata Inc
VATA
3
https://www.sec.gov/Archives/edgar/data/1034397/0000950149-99-002215-index.htm
AsiaInfo Holdings Inc
ASIA
4
https://www.sec.gov/Archives/edgar/data/1100969/0000950109-99-004557-index.htm
Register.com Inc
RCOM
5
https://www.sec.gov/Archives/edgar/data/1091284/0000950116-00-001718-index.htm
Prime Response Inc
PRME
Switchboard Inc
SWBD
net.Genesis Corp
NTGX
Onvia.com Inc
ONVI
9
https://www.sec.gov/Archives/edgar/data/1100917/0001032210-99-001778-index.htm
Niku Corp
NIKU
10
https://www.sec.gov/Archives/edgar/data/1076641/0000891618-99-005800-index.htm
Avenue A Inc
AVEA
Hotel Reservations Network Inc
ROOM
Digitalthink Inc
DTHK
Carrier1 International SA
CONE
Inforte Corp
INFT
Gigamedia Ltd
GIGM
Choice One Communications
CWON
Interactive Investor Intl PLC
IINV
VarsityBooks.com Inc
VSTY
19
https://www.sec.gov/Archives/edgar/data/1069502/0000950133-99-003239-index.htm
LendingTree Inc
TREE
20
https://w

In [252]:
df_5 = get_data(list_of_df[5].reset_index(drop=True))

Mediaplex Inc
MPLX
0
https://www.sec.gov/Archives/edgar/data/1552000/0001193125-12-292127-index.htm
SciQuest.com Inc
SQST
CacheFlow Inc
CFLO
Web Street Inc
WEBS
Terra Networks(Telefonica SA)
TRRA
Retek Inc(HNC Software Inc)
RETK
Rainmaker Systems Inc
RMKR
Korea Thrunet Co
KOREA
Virata Corp
VRTA
Quintus Corp
QNTS
Netcreations Inc
NTCR
SonicWALL Inc
SNWL
11
https://www.sec.gov/Archives/edgar/data/1093885/0001012870-00-000921-index.htm
ASD Systems Inc
ASDS
ibasis Inc
IBAS
Next Level Communications
NXTV
Netzee Inc
NETZ
Expedia Inc
EXPE
WebVan Group Inc
WBVN
Cobalt Networks Inc
COBT
Tickets.com
TIXX
19
https://www.sec.gov/Archives/edgar/data/1038083/0000892569-99-001628-index.htm
Be Free Inc
BFRE
Akamai Technologies Inc
AKAM
21
https://www.sec.gov/Archives/edgar/data/1086222/0000950135-00-004319-index.htm
Cavion Technologies Inc
CAVN
Gaiam Inc
GAIA
23
https://www.sec.gov/Archives/edgar/data/1089872/0001021408-01-502367-index.htm
Viador Inc
VIAD
InterTrust Technologies Corp
ITRU
Sycamore Net

In [254]:
df_6 = get_data(list_of_df[6].reset_index(drop=True))

Quest Software Inc
QSFT
0
https://www.sec.gov/Archives/edgar/data/1088033/0000892569-00-000249-index.htm
Active Software Inc
ASWX
Red Hat Inc
RHAT
Mortgage.com Inc
MDCM
InterWorld Corp
INTW
Hotjobs Com LTD
HOTJ
US Interactive Inc
USIT
Interactive Pictures Corp
IPIX
7
https://www.sec.gov/Archives/edgar/data/1088022/0000950144-00-003313-index.htm
Cobalt Group Inc
CBLT
Internet Capital Group
ICGE
Internet Gold-Golden Lines Ltd
IGLD
Tumbleweed Communications Corp
TMWD
11
https://www.sec.gov/Archives/edgar/data/1022509/0000912057-00-031540-index.htm
Homestore.com Inc
HOMS
Quotesmith.com Inc
QUOT
13
https://www.sec.gov/Archives/edgar/data/1115128/0001193125-14-030698-index.htm
Internet Initiative Japan Co
IIJI
SplitRock Services Inc
SPLT
1-800-Flowers.com Inc
FLWS
16
https://www.sec.gov/Archives/edgar/data/1084869/0001047469-99-021779-index.htm
Watchguard Technologies Inc
WGRD
17
https://www.sec.gov/Archives/edgar/data/1062019/0001032210-00-000049-index.htm
Net2Phone Inc
NTOP
18
https://www.

In [256]:
df_7 = get_data(list_of_df[7].reset_index(drop=True))

Phone.com Inc
PHCM
Litronic Inc
LTNX
drkoop.com Inc
KOOP
BackWeb Technologies Ltd
BWEB
3
https://www.sec.gov/Archives/edgar/data/1082064/0000891618-00-001882-index.htm
F5 Networks Inc
FFIV
4
https://www.sec.gov/Archives/edgar/data/1048695/0001047469-99-035137-index.htm
Wit Capital Group Inc
WITC
Online Resources & Commun
ORCC
Network Access Solutions Corp
NASC
7
https://www.sec.gov/Archives/edgar/data/1081665/0000928385-99-003671-index.htm
High Speed Access Corp
HSAC
8
https://www.sec.gov/Archives/edgar/data/1764013/000119312520103101/0001193125-20-103101-index.htm
iXL Enterprises Inc
IIXL
Edgar Online Inc
EDGR
10
https://www.sec.gov/Archives/edgar/data/1080224/0000950123-99-002764-index.htm
ZipLink Inc
ZIPL
barnesandnoble.com inc
BNBN
Star Media Network Inc
STRM
Juno Online Services Inc
JWEB
DLJdirect(Donaldson Lufkin)
DIR
Fashionmall.com
FASH
CAIS Internet Inc
CAIS
@plan.inc
APLN
OneSource Information Services
ONES
eToys Inc
ETYS
Redback Networks Inc
RBAK
21
https://www.sec.gov/Archi

In [258]:
df_8 = get_data(list_of_df[8].reset_index(drop=True))

MarketWatch.com Inc
MKTW
Artificial Life Inc
ALIF
1
https://www.sec.gov/Archives/edgar/data/1070361/0000950135-98-005279-index.htm
Concur Technologies Inc
CNQR
2
https://www.sec.gov/Archives/edgar/data/1066026/0000891618-99-002786-index.htm
InfoSpace.com
INSP
3
https://www.sec.gov/Archives/edgar/data/1609550/0001047469-18-007469-index.htm
Internet America Inc
GEEK
AboveNet Communications Inc
ABOV
Xoom.com Inc
XMCM
uBID
UBID
Ticketmaster Online-CitySearch
TMCS
E-Tek Dynamics Inc
ETEK
theglobe.com inc
TGLO
10
https://www.sec.gov/Archives/edgar/data/1066684/0000895345-99-000194-index.htm
EarthWeb Inc
EWBX
eBay Inc
EBAY
12
https://www.sec.gov/Archives/edgar/data/1065088/0001012870-99-000834-index.htm
Global Crossing Ltd
GBLC
24/7 Media Inc
TFSM
14
https://www.sec.gov/Archives/edgar/data/1062195/0000950146-99-000051-index.htm
Digital River Inc
DRIV
15
https://www.sec.gov/Archives/edgar/data/1062530/0001047469-98-042234-index.htm
Pilot Network Services
PILT
GeoCities
GCTY
Cyberian Outpost
CO

In [260]:
df_9 = get_data(list_of_df[9].reset_index(drop=True))

Spyglass
SPYG
UUNet Technologies
UUNT
Transaction Systems
TSAI
Infosafe Systems
ISFEA
Netcom On-Line Communications
NETC
Investment Technology Group
ITGI
Navarre
NAVR
Macromedia
MACR
FutureMedia
FMDAY
Micro Warehouse
MWHS
Olicom
OLCMF
America Online
AMER
Valassis Communications
VCI
Cisco Systems
CSCO


,IPO name,Offer date,SIC,Ticker,Unit,Offer price,S1Link
0,Spyglass,19950627,7372,SPYG,1,17.000,
1,UUNet Technologies,19950525,7375,UUNT,1,14.000,
2,Transaction Systems,19950223,7372,TSAI,1,15.000,
3,Infosafe Systems,19950118,5045,ISFEA,2,5.000,
4,Netcom On-Line Communications,19941214,5045,NETC,1,13.000,
5,Investment Technology Group,19940504,7379,ITGI,1,13.000,
6,Navarre,19931216,5045,NAVR,1,5.625,
7,Macromedia,19931213,7371,MACR,1,12.000,
8,FutureMedia,19930819,7812,FMDAY,2,5.000,
9,Micro Warehouse,19921210,5961,MWHS,1,18.000,


In [263]:
merged_df = pd.concat([df_0, df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9])

In [268]:
merged_df = merged_df.reset_index(drop=True)

In [269]:
merged_df.to_csv("output_sec-api.csv")

In [289]:
results = pd.read_csv("output_sec-api.csv")

In [300]:
import numpy as np
results = results.dropna().reset_index(drop=True)
results

,Unnamed: 0,IPO name,Offer date,SIC,Ticker,Unit,Offer price,S1Link
0,1,Anaplan Inc,20181012,7376,PLAN,1,17.0,"<a href=""https://www.sec.gov/Archives/edgar/data/1540755/0001193125-18-274158-index.htm"">Anaplan Inc</a>"
1,2,Elastic NV,20181005,7379,ESTC,1,36.0,"<a href=""https://www.sec.gov/Archives/edgar/data/1707753/0001193125-18-266861-index.htm"">Elastic NV</a>"
2,3,Upwork Inc,20181003,7374,UPWK,1,15.0,"<a href=""https://www.sec.gov/Archives/edgar/data/1627475/0001193125-18-267594-index.htm"">Upwork Inc</a>"
3,4,SVMK Inc,20180926,7374,SVMK,1,12.0,"<a href=""https://www.sec.gov/Archives/edgar/data/1739936/0001193125-18-261892-index.htm"">SVMK Inc</a>"
4,6,Eventbrite Inc,20180920,7372,EB,1,23.0,"<a href=""https://www.sec.gov/Archives/edgar/data/1475115/0001193125-18-255960-index.htm"">Eventbrite Inc</a>"
5,7,Avalara Inc,20180615,7372,AVLR,1,24.0,"<a href=""https://www.sec.gov/Archives/edgar/data/1348036/0001193125-19-164571-index.htm"">Avalara Inc</a>"
6,8,GreenSky Inc,20180524,7389,GSKY,1,23.0,"<a href=""https://www.sec.gov/Archives/edgar/data/1712923/0000930413-18-001566-index.htm"">GreenSky Inc</a>"
7,9,Pluralsight Inc,20180517,7370,PS,1,15.0,"<a href=""https://www.sec.gov/Archives/edgar/data/1725579/0001628280-19-002420-index.htm"">Pluralsight Inc</a>"
8,10,Dropbox Inc,20180323,7375,DBX,1,21.0,"<a href=""https://www.sec.gov/Archives/edgar/data/1467623/0001193125-18-055809-index.htm"">Dropbox Inc</a>"
9,11,Bandwidth.com Inc,20171110,7375,BAND,1,20.0,"<a href=""https://www.sec.gov/Archives/edgar/data/1514416/0001193125-17-309873-index.htm"">Bandwidth.com Inc</a>"


In [304]:
results = results.drop(['Unnamed: 0'], axis=1)

In [294]:
# convert links to html 
for i in results.iterrows():
    index = i[0]
    name = i[1]['IPO name']
    s1link = i[1]['S1Link']
    if s1link != "":
        hyperlink_format = '<a href="{}">{}</a>'.format(s1link, name)
        results.iloc[index, 7] = hyperlink_format
    

In [305]:
results.to_html('index.html',
    render_links=True,
    escape=False,
    index=False
)